In [1]:
import torch
import torchvision.transforms as transforms
from torchvision.datasets import ImageFolder
from torch.utils.data import DataLoader, random_split
import torch.nn as nn
import torch.optim as optim
from torchvision import models
from sklearn.metrics import accuracy_score

# Paths
data_dir = "dcn/"

# Transformations
transform = transforms.Compose([
    transforms.Resize((128, 128)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
])

# Load Dataset
dataset = ImageFolder(root=data_dir, transform=transform)

# Split Dataset
train_size = int(0.8 * len(dataset))
val_size = len(dataset) - train_size
train_dataset, val_dataset = random_split(dataset, [train_size, val_size])

# Data Loaders
train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=32, shuffle=False)

# Model
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = models.resnet18(pretrained=True)
model.fc = nn.Linear(model.fc.in_features, len(dataset.classes))  # Update the final layer
model = model.to(device)

# Loss and Optimizer
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)



/home/vamsi/miniconda3/envs/sr3/lib/python3.11/site-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/home/vamsi/miniconda3/envs/sr3/lib/python3.11/site-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet18_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet18_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


In [2]:
# Training Loop
epochs = 10
for epoch in range(epochs):
    model.train()
    train_loss = 0.0
    train_correct = 0
    train_total = 0

    for inputs, labels in train_loader:
        inputs, labels = inputs.to(device), labels.to(device)

        optimizer.zero_grad()
        outputs = model(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        train_loss += loss.item()
        _, preds = torch.max(outputs, 1)
        train_correct += (preds == labels).sum().item()
        train_total += labels.size(0)

    train_accuracy = train_correct / train_total

    # Validation
    model.eval()
    val_loss = 0.0
    val_correct = 0
    val_total = 0

    with torch.no_grad():
        for inputs, labels in val_loader:
            inputs, labels = inputs.to(device), labels.to(device)

            outputs = model(inputs)
            loss = criterion(outputs, labels)

            val_loss += loss.item()
            _, preds = torch.max(outputs, 1)
            val_correct += (preds == labels).sum().item()
            val_total += labels.size(0)

    val_accuracy = val_correct / val_total

    # Print Metrics
    print(f"Epoch [{epoch + 1}/{epochs}]")
    print(f"Train Loss: {train_loss / len(train_loader):.4f}, Train Accuracy: {train_accuracy:.4f}")
    print(f"Val Loss: {val_loss / len(val_loader):.4f}, Val Accuracy: {val_accuracy:.4f}")

# Save the Model
torch.save(model.state_dict(), "classifier_model4.pth")


Epoch [1/10]
Train Loss: 1.0183, Train Accuracy: 0.6326
Val Loss: 0.7832, Val Accuracy: 0.7117
Epoch [2/10]
Train Loss: 0.6595, Train Accuracy: 0.7672
Val Loss: 0.7898, Val Accuracy: 0.7196
Epoch [3/10]
Train Loss: 0.5506, Train Accuracy: 0.8113
Val Loss: 0.7534, Val Accuracy: 0.7427
Epoch [4/10]
Train Loss: 0.4604, Train Accuracy: 0.8395
Val Loss: 0.7270, Val Accuracy: 0.7525
Epoch [5/10]
Train Loss: 0.4224, Train Accuracy: 0.8507
Val Loss: 0.5600, Val Accuracy: 0.8052
Epoch [6/10]
Train Loss: 0.3534, Train Accuracy: 0.8779
Val Loss: 0.7366, Val Accuracy: 0.7616
Epoch [7/10]
Train Loss: 0.3155, Train Accuracy: 0.8895
Val Loss: 0.6213, Val Accuracy: 0.8019
Epoch [8/10]
Train Loss: 0.2532, Train Accuracy: 0.9117
Val Loss: 0.5677, Val Accuracy: 0.8286
Epoch [9/10]
Train Loss: 0.1860, Train Accuracy: 0.9350
Val Loss: 0.8338, Val Accuracy: 0.7370
Epoch [10/10]
Train Loss: 0.1691, Train Accuracy: 0.9453
Val Loss: 0.6301, Val Accuracy: 0.8179
